In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
npDeathWoman = np.array(pd.read_excel('../../data/여성사망률추이.xlsx').set_index(['연도']).iloc[:71,:100])
npDeathMan = np.array(pd.read_excel('../../data/남성사망률추이.xlsx').set_index(['연도']).iloc[:71,:100])
npPassDoctor = np.around(np.array((pd.read_excel('../../data/데이터정리3.xlsx'))[['년도','의대졸합격/남', '의대졸합격/여', '의전졸합격/남', '의전졸합격/여', '불합합격/남', '불합합격/여']].iloc[27:,1:7])).T
npDoctorAgeData = np.array(pd.read_excel('../../data/의사연령별분포.xlsx'))
npFutureData = pd.read_excel('추계인구.xlsx')

In [7]:
npDeathWoman #각 연도별이 한 묶음

array([[3.3770e-02, 4.8400e-03, 2.4400e-03, ..., 3.1921e-01, 3.3806e-01,
        3.5734e-01],
       [3.2840e-02, 4.7600e-03, 2.3800e-03, ..., 3.1598e-01, 3.3480e-01,
        3.5407e-01],
       [3.1940e-02, 4.6800e-03, 2.3300e-03, ..., 3.1276e-01, 3.3154e-01,
        3.5081e-01],
       ...,
       [6.5000e-04, 5.0000e-05, 4.0000e-05, ..., 2.6318e-01, 2.9301e-01,
        3.2446e-01],
       [6.2000e-04, 4.0000e-05, 4.0000e-05, ..., 2.6182e-01, 2.9163e-01,
        3.2306e-01],
       [5.9000e-04, 4.0000e-05, 3.0000e-05, ..., 2.6068e-01, 2.9049e-01,
        3.2193e-01]])

In [8]:
npPassDoctor

array([[ 932.,  963.,  953., 1030., 1026., 1220., 1187., 1032., 1242.,
        1656., 2241., 2157., 2194., 1983., 2195., 2116., 2400., 2282.,
        1602., 1321., 2274., 1956., 2006., 1961., 1676., 2205., 1947.,
        2479., 2128., 1992., 1881., 2440., 2020., 1666., 1610., 1628.,
        1072., 1078., 1058., 1001.,  931.,  970., 1200., 1165., 1165.,
        1165., 1165., 1165., 1165., 1165., 1165., 1165., 1165., 1165.,
        1165., 1165., 1165., 1165., 1165., 1165., 1165., 1165., 1165.,
        1165., 1165., 1165., 1165., 1165., 1165., 1165., 1165.],
       [ 155.,  154.,  156.,  178.,  132.,  111.,  150.,  188.,  181.,
         302.,  304.,  391.,  526.,  493.,  535.,  550.,  616.,  522.,
         354.,  337.,  544.,  551.,  620.,  596.,  724.,  822.,  769.,
        1044.,  940., 1200., 1039., 1270., 1112.,  776.,  641.,  580.,
         434.,  475.,  439.,  468.,  424.,  453.,  586.,  569.,  569.,
         569.,  569.,  569.,  569.,  569.,  569.,  569.,  569.,  569.,
         569

In [2]:
if torch.cuda.is_available():
    cuda = torch.device('cuda:1')
    cuda

In [3]:
yearSize = 71
oldSize = 100
ageModelSize = 6
RetireModelSize = 2

# 기본데이터 텐서 정의

In [73]:
# 외부데이터
tsPassDoctorMan = torch.tensor([npPassDoctor[0],npPassDoctor[2],npPassDoctor[4]]).to(device=cuda)
tsPassDoctorWoman = torch.tensor([npPassDoctor[1],npPassDoctor[3],npPassDoctor[5]]).to(device=cuda)
tsDeathWoman = torch.tensor(npDeathWoman).to(device=cuda)
tsDeathMan = torch.tensor(npDeathMan).to(device=cuda)
tsDoctorAgeData = torch.tensor(npDoctorAgeData).to(device=cuda)

# 계산데이터
tsBasic = torch.zeros(oldSize).to(device=cuda)
tsNewPerson = torch.zeros([yearSize,2,oldSize]).to(device=cuda)
tsResultPerson = torch.zeros([yearSize,2,oldSize]).to(device=cuda)
tsAgeTuningSet = torch.zeros([ageModelSize,3]).to(device=cuda)
tsRetireTuningSet = torch.zeros([RetireModelSize,3]).to(device=cuda)

# 연령분포 모델 텐서 설계

In [61]:
npBasic = np.zeros(100)
npAgeDistribution = np.zeros([30000,100])

rateArray = np.arange(0,1,0.001)
endArray = np.arange(20,50,1)

for i in range(1000):
    if i==0:
        continue
    for j in range(30):
        for k in range(100):
            npBasic[k] = (rateArray[i]**(k-endArray[j]))-1
            
        npSum = np.sum(npBasic)
        
        if npSum!=0:
            npBasic = npBasic / npSum
        else:
            npBasic = 0
        
        npAgeDistribution[(i*j)+j] = npBasic
        
tsAgeDistribution = torch.from_numpy(npAgeDistribution).to(device=cuda)

tsAgeDistribution

tensor([[-7.0476e-03, -6.6919e-03, -6.3365e-03,  ...,  2.5854e-02,
          2.6177e-02,  2.6500e-02],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 9.9900e-01,  9.9900e-04,  9.9900e-07,  ..., -9.9900e-64,
         -9.9900e-64, -9.9900e-64],
        ...,
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00]], device='cuda:1', dtype=torch.float64)

In [66]:
tsAgeDistribution.size()

torch.Size([30000, 100])

In [77]:
tsPassDoctorMan
tsPassDoctorWoman

tensor([[ 155.,  154.,  156.,  178.,  132.,  111.,  150.,  188.,  181.,  302.,
          304.,  391.,  526.,  493.,  535.,  550.,  616.,  522.,  354.,  337.,
          544.,  551.,  620.,  596.,  724.,  822.,  769., 1044.,  940., 1200.,
         1039., 1270., 1112.,  776.,  641.,  580.,  434.,  475.,  439.,  468.,
          424.,  453.,  586.,  569.,  569.,  569.,  569.,  569.,  569.,  569.,
          569.,  569.,  569.,  569.,  569.,  569.,  569.,  569.,  569.,  569.,
          569.,  569.,  569.,  569.,  569.,  569.,  569.,  569.,  569.,  569.,
          569.],
        [   0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
            0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
            0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
            0.,    0.,   87.,  304.,  322.,  416.,  702.,  778.,  721.,  696.,
          669.,  709.,  518.,  503.,  503.,  503.,  503.,  503.,  503.,  503.,
          503.,  503.,  503.,  503.

# 은퇴율 모델 텐서 설계

In [75]:
npBasic = np.zeros(100)
npRetireRate = np.zeros([100,1000,50,100])

rate1Array = np.arange(1,2,0.001)
rate2Array = np.arange(0,1,0.01)
startArray = np.arange(30,80,1)

for i in range(100):
    for j in range(1000):
        if i==0:
            continue
    for k in range(50):
        for m in range(100):
            a = (rate1Array[j]**(m-startArray[k]))-1
            b = rate2Array[i]/((rate1Array[j]**(100-startArray[k]))-1)
            npBasic[m] = a*b
            
        npRetireRate[i][j] = npBasic
        
tsRetireRate = torch.from_numpy(npRetireRate).to(device=cuda)
tsRetireRate.size()

torch.Size([100, 1000, 50, 100])

# 신규인원 생성

In [46]:
def newPerson():
    tsNewPerson[0] = tsPassDoctorMan@[tsAgeDistribution,tsAgeDistribution,tsAgeDistribution]
    

In [64]:
tsPassDoctor.size()

torch.Size([71, 6])

In [47]:
%time newPerson()

CPU times: user 46.9 ms, sys: 5.09 ms, total: 52 ms
Wall time: 50 ms


In [105]:
tsApplyAgeModel[0]

tensor([[0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 6.9431e-06, 3.4711e-06, 1.7351e-06, 8.6714e-07,
         4.3315e-07, 2.1615e-07, 1.0765e-07, 5.3402e-08, 2.6277e-08, 1.2715e-08,
         5.9335e-09, 2.5429e-09, 8.4765e-10, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e

In [55]:
a = np.array([[1,2],[3,4],[5,6],[7,8]])
b = np.array([[[1,2,3],[4,5,6]],[[7,8,9],[10,11,12]],[[13,14,15],[16,17,18]]])
c = a@b
print(np.shape(a))
print(np.shape(b))
print(np.shape(c))
c

(4, 2)
(3, 2, 3)
(3, 4, 3)


array([[[  9,  12,  15],
        [ 19,  26,  33],
        [ 29,  40,  51],
        [ 39,  54,  69]],

       [[ 27,  30,  33],
        [ 61,  68,  75],
        [ 95, 106, 117],
        [129, 144, 159]],

       [[ 45,  48,  51],
        [103, 110, 117],
        [161, 172, 183],
        [219, 234, 249]]])

In [80]:
a = np.array([3,5])
b = np.array([[5,6],[7,8]])
a@b

array([50, 58])